In [ ]:
import h5py
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import gseapy as gp
import os
print(ad.__version__)

In [ ]:
# Let's load the file using anndata's read_h5ad function
adata = ad.read_h5ad("f9986ff4-66fc-4265-8124-806f0c6313df.h5ad")

# Print basic information about the dataset
print(f"AnnData object with n_obs × n_vars = {adata.n_obs} × {adata.n_vars}")

# The number of genes (variables) is:
print(f"Number of genes: {adata.n_vars}")

# You can also view the first few gene names
print("\nFirst 10 gene names:")
print(adata.var_names[:10])

In [ ]:
# general information about the dataset
adata

In [ ]:
# cell type annotations
adata.obs['cell_type'].unique()

In [ ]:
# Get a subset of cells and genes for visualization
n_cells = 10  # Number of cells to show
n_genes = 20  # Number of genes to show

# Create a DataFrame with expression values for selected cells and genes
expression_df = pd.DataFrame(
    adata.X[:n_cells, :n_genes].toarray() if isinstance(adata.X, csr_matrix) else adata.X[:n_cells, :n_genes],
    index=adata.obs_names[:n_cells],
    columns=adata.var_names[:n_genes]
)

# Add the cell type as an additional column
expression_df['cell_type'] = adata.obs['cell_type'][:n_cells].values

# Display the DataFrame
print("Expression matrix with cell type labels:")
print(expression_df)

# Alternative visualization: heatmap with cell types as labels

# Create a figure with larger size
plt.figure(figsize=(16, 8))

# Create a separate DataFrame for plotting without the cell type column
plot_df = expression_df.drop(columns=['cell_type'])

# Plot the heatmap
sns.heatmap(plot_df, cmap="viridis", yticklabels=True)

# Create a second y-axis for cell type labels
ax2 = plt.twinx()
ax2.set_yticks(np.arange(0.5, len(expression_df)))
ax2.set_yticklabels(expression_df['cell_type'], fontsize=8)
ax2.set_ylim([0, len(expression_df)])

plt.title('Gene Expression by Cell Type')
plt.tight_layout()
plt.show()

In [ ]:
# First, let's see what cell types are available
print("Available cell types:")
cell_types = adata.obs['cell_type'].unique()
print(cell_types)
print(f"\nTotal number of cell types: {len(cell_types)}")

# Let's also see the counts for each cell type
print("\nCell type counts:")
print(adata.obs['cell_type'].value_counts())

# Use the entire dataset for differential expression analysis
print("\nUsing all cell types for DEA")
adata_for_dea = adata.copy()
print(f"Dataset for DEA: {adata_for_dea.n_obs} cells × {adata_for_dea.n_vars} genes")

# Run differential expression analysis using all cell types
sc.tl.rank_genes_groups(
    adata_for_dea, 
    groupby='cell_type',
    method='wilcoxon',  # You can also use 't-test' or 'logreg'
    key_added='rank_genes_groups',
    pts=True            # Calculate percentage of cells expressing genes
)

# Display the results
print("\nTop differentially expressed genes:")
sc.pl.rank_genes_groups(adata_for_dea, n_genes=10, sharey=False)

# Get the results as a DataFrame for easier viewing
result = sc.get.rank_genes_groups_df(adata_for_dea, group=None)
print("\nTop 20 DEGs (all groups):")
print(result.head(20))

# Get results for each cell type
for cell_type in cell_types:
    print(f"\nTop 10 DEGs for {cell_type}:")
    result_specific = sc.get.rank_genes_groups_df(adata_for_dea, group=cell_type)
    print(result_specific.head(10)[['names', 'scores', 'logfoldchanges', 'pvals', 'pvals_adj']])

# Summary of significant genes (adjusted p-value < 0.05)
print(f"\nSummary of significant DEGs (padj < 0.05):")
for cell_type in cell_types:
    result_specific = sc.get.rank_genes_groups_df(adata_for_dea, group=cell_type)
    significant = result_specific[result_specific['pvals_adj'] < 0.05]
    print(f"{cell_type}: {len(significant)} significant DEGs")

In [ ]:
# Improved volcano plot function for all cell types
def plot_volcano_all_genes(adata, group_name=None, title_suffix=""):
    # Get all DE results
    if group_name:
        de_results = sc.get.rank_genes_groups_df(adata, group=group_name)
        title = f'Volcano Plot - {group_name} {title_suffix}'
    else:
        de_results = sc.get.rank_genes_groups_df(adata, group=None)
        title = f'Volcano Plot - All Groups {title_suffix}'
    
    # Remove any NaN values
    de_results = de_results.dropna(subset=['logfoldchanges', 'pvals_adj'])
    
    # Calculate -log10 p-values
    neg_log_pvals = -np.log10(de_results['pvals_adj'])
    
    # Create the plot
    plt.figure(figsize=(12, 8))
    
    # Color points based on significance and fold change
    significant = de_results['pvals_adj'] < 0.05
    high_fc = np.abs(de_results['logfoldchanges']) > 0.5  # Adjust threshold as needed
    
    # Plot non-significant genes in gray
    non_sig = ~significant
    plt.scatter(de_results.loc[non_sig, 'logfoldchanges'], 
               neg_log_pvals[non_sig], 
               alpha=0.5, color='gray', s=10, label='Not significant')
    
    # Plot significant genes with low fold change in blue
    sig_low_fc = significant & ~high_fc
    if sig_low_fc.any():
        plt.scatter(de_results.loc[sig_low_fc, 'logfoldchanges'], 
                   neg_log_pvals[sig_low_fc], 
                   alpha=0.7, color='blue', s=20, label='Significant (low FC)')
    
    # Plot significant genes with high fold change in red
    sig_high_fc = significant & high_fc
    if sig_high_fc.any():
        plt.scatter(de_results.loc[sig_high_fc, 'logfoldchanges'], 
                   neg_log_pvals[sig_high_fc], 
                   alpha=0.8, color='red', s=30, label='Significant (high FC)')
    
    # Add threshold lines
    plt.axhline(y=-np.log10(0.05), color='red', linestyle='--', alpha=0.7, label='p-adj = 0.05')
    plt.axvline(x=0.5, color='orange', linestyle='--', alpha=0.7)
    plt.axvline(x=-0.5, color='orange', linestyle='--', alpha=0.7, label='|FC| = 0.5')
    
    plt.xlabel('Log Fold Change')
    plt.ylabel('-Log10 Adjusted P-value')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add some stats
    n_significant = significant.sum()
    n_total = len(de_results)
    plt.text(0.02, 0.98, f'{n_significant}/{n_total} significant genes', 
             transform=plt.gca().transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return de_results

# Create volcano plots for each cell type
print("\nCreating volcano plots for each cell type:")
for cell_type in cell_types:
    print(f"Volcano plot for {cell_type}:")
    volcano_results = plot_volcano_all_genes(adata_for_dea, group_name=cell_type, 
                                            title_suffix=f"vs. all other cell types")